In [265]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt

In [266]:
def Cumulative_Return(df):
    cum_return = (df.iloc[-1][0]-df.iloc[1][0])/df.iloc[1][0] # (Current_Price-Initial_Price)/Initial_Price
    return cum_return

def Volatility(data, N = 252):
    sd = data.std() * np.sqrt(N) # Daily Return Annualized
    return sd

In [267]:
def Sharpe_ratio(data, N=252, rf=0.00):
    adjusted_mean = data.mean() * N - rf
    sd = data.std() * np.sqrt(N)  # Annualized standard deviation
    return adjusted_mean / sd


def Sortino_ratio(data, N=252, rf=0.00):
    mean = data.mean() * N - rf
    std_neg = data[data < 0].std() * np.sqrt(N)  # Only negative standard deviation
    return mean / std_neg


def Max_dd(data):
    r = data.add(1).cumprod()  # Reverting back to ratios
    dd = r.div(r.cummax()).sub(1)  # Ratio by the current maximum and -1
    max_dd = dd.min()
    return max_dd

In [268]:
start = dt.datetime(2010, 1, 1)
end = dt.datetime(2023, 5, 1)

tickers_ind = ['^GSPC','^DJI','^IXIC','^NYA','^XAX']
tickers_eq = ['AAPL','MSFT','AMZN','NVDA','GOOGL']

cum_returns_ind = []
vol_ind = []
sharpe_ind = []
sortino_ind = []
dd_ind = []

cum_returns_eq = []
vol_eq = []
sharpe_eq = []
sortino_eq = []
dd_eq = []

for _ in tickers_ind:
    df = yf.download(_, start, end)
    df = df[['Adj Close']]
    df_adjusted = df.pct_change().dropna()
    df_adjusted = df_adjusted[['Adj Close']]
    cum_returns_ind.append(Cumulative_Return(df))
    vol_ind.append(Volatility(df_adjusted['Adj Close'].to_numpy()))
    sharpe_ind.append(Sharpe_ratio(df_adjusted['Adj Close'].to_numpy(),rf = 3.7/100)) #Risk-Free rate = 3.7%
    sortino_ind.append(Sortino_ratio(df_adjusted['Adj Close'].to_numpy(),rf = 3.7/100))
    dd_ind.append(Max_dd(df_adjusted['Adj Close']))

for _ in tickers_eq:
    df = yf.download(_, start, end)
    df = df[['Adj Close']]
    df_adjusted = df.pct_change().dropna()
    df_adjusted = df_adjusted[['Adj Close']]
    cum_returns_eq.append(Cumulative_Return(df))
    vol_eq.append(Volatility(df_adjusted['Adj Close'].to_numpy()))
    sharpe_eq.append(Sharpe_ratio(df_adjusted['Adj Close'].to_numpy(),rf = 3.7/100)) #Risk-Free rate = 3.7%
    sortino_eq.append(Sortino_ratio(df_adjusted['Adj Close'].to_numpy(),rf = 3.7/100))
    dd_eq.append(Max_dd(df_adjusted['Adj Close']))
    


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [269]:
df_ind = pd.DataFrame(np.column_stack([cum_returns_ind, vol_ind, sharpe_ind, sortino_ind, dd_ind]),
                   columns=['Cumulative Returns', 'Volatility', 'Sharpe Ratio','Sortino Ratio','Max DropDown'])
df_eq = pd.DataFrame(np.column_stack([cum_returns_eq, vol_eq, sharpe_eq, sortino_eq, dd_eq]),
                   columns=['Cumulative Returns', 'Volatility', 'Sharpe Ratio','Sortino Ratio','Max DropDown'])

In [270]:
df_ind.insert(0,'Index',tickers_ind)


In [271]:
df_eq.insert(0,'Equity',tickers_eq)

In [272]:
df_ind

,Index,Cumulative Returns,Volatility,Sharpe Ratio,Sortino Ratio,Max DropDown
0,^GSPC,2.668638,0.177607,0.432630,0.529764,-0.339250
1,^DJI,2.225321,0.171792,0.382973,0.462916,-0.370862
2,^IXIC,4.295849,0.206024,0.532287,0.672364,-0.363953
3,^NYA,1.113685,0.175491,0.199692,0.243946,-0.381142
4,^XAX,1.257893,0.195960,0.223779,0.285101,-0.535116


In [273]:
df_eq

,Equity,Cumulative Returns,Volatility,Sharpe Ratio,Sortino Ratio,Max DropDown
0,AAPL,25.038410,0.285959,0.871522,1.217357,-0.437972
1,MSFT,12.002119,0.261356,0.727317,1.016921,-0.371485
2,AMZN,14.658178,0.331749,0.678852,0.969564,-0.561453
3,NVDA,63.479655,0.447385,0.842415,1.241592,-0.663351
4,GOOGL,5.873999,0.272936,0.530283,0.740847,-0.443201
